In [15]:
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import string
import re
from collections import Counter

import nltk
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow import keras
from keras.preprocessing.text import one_hot


In [16]:
data = pd.read_csv('train.csv')
data.head(5)

,rating,reviewText,summary
0,3,Plot Storyline: 5 StarsThis novel accomplished...,3 1/4 Stars
1,3,I did not like how EL ended this one. I don't ...,"It was going great, then just.... ended"
2,5,I love how old fashioned this family is - they...,LOVED ALL 4!
3,5,I loved this story - It's about two friends wh...,friends make the best lovers
4,1,"In the Dark Lands, a virus killed all possibil...",Blatantly sexist and homophobic


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   rating      9000 non-null   int64 
 1   reviewText  9000 non-null   object
 2   summary     9000 non-null   object
dtypes: int64(1), object(2)
memory usage: 211.1+ KB


In [20]:
review = []

my_stopwords = set(nltk.corpus.stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

vocabulary = {}

review_size = []

def tokenize(text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return nltk.tokenize.word_tokenize(nopunct)


for i in range(len(data['rating'])):
    review.append(data['reviewText'][i] +' '+ data['summary'][i])

    
counts = Counter()

for i in range(len(review)):
    r = []
    tokens = tokenize(review[i])
    for t in tokens:
        if t not in my_stopwords and len(t) > 1:
            r.append(lemmatizer.lemmatize(t))
            
    review[i] = r
    counts.update(r)
    review_size.append(len(r))
    #print(review[i])
    
 
data['review'] = review


vocab_size = 20000
encoded = []
for r in review:
    encoded_review = [one_hot(d, vocab_size) for d in r]
    encoded.append(encoded_review)
    
print(review_size)
print(review[-1])
print(counts)

[298, 34, 28, 36, 137, 16, 34, 19, 14, 53, 10, 11, 48, 84, 270, 275, 19, 108, 64, 48, 43, 19, 87, 32, 104, 97, 47, 141, 130, 48, 61, 42, 21, 31, 78, 14, 23, 18, 55, 292, 21, 41, 15, 19, 90, 18, 17, 20, 33, 106, 13, 48, 30, 28, 29, 37, 28, 32, 28, 20, 19, 14, 14, 15, 40, 49, 125, 10, 39, 73, 23, 24, 15, 19, 39, 69, 23, 93, 17, 36, 9, 50, 24, 25, 62, 356, 26, 30, 89, 42, 48, 27, 11, 76, 64, 18, 27, 15, 49, 48, 84, 46, 80, 160, 237, 42, 46, 167, 237, 21, 13, 26, 16, 26, 23, 98, 30, 175, 36, 26, 111, 23, 16, 19, 110, 67, 80, 27, 12, 135, 43, 81, 13, 14, 24, 36, 16, 147, 15, 10, 17, 121, 25, 29, 17, 132, 16, 30, 133, 18, 105, 83, 142, 25, 11, 11, 10, 61, 85, 32, 17, 21, 66, 35, 92, 58, 13, 8, 69, 20, 42, 23, 208, 33, 3, 62, 20, 41, 52, 35, 41, 65, 19, 21, 15, 25, 268, 13, 51, 20, 16, 14, 16, 18, 103, 12, 14, 23, 67, 69, 27, 11, 26, 12, 43, 9, 35, 15, 84, 44, 27, 15, 32, 16, 11, 72, 14, 11, 87, 15, 16, 12, 34, 15, 66, 135, 36, 20, 21, 13, 30, 56, 95, 14, 50, 44, 28, 28, 97, 28, 58, 5, 42, 22